In [1]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

#filename = "math sample for API testing.csv"
filename = "Big Math Subset.csv"

In [172]:
def process_file(filename, tagging_df):
    parsed_data = []
    error_data = []
    component_holding = []
    initial_components = True
    content_focus_components_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(1):
            l = reader.next()    #########See if I can simplify this

        for line in reader:
            error_test = False
            component_list = line['Component'].split("; ")
            content_focus_list = line['Content Focus'].split("; ")
            #first pass, only process those items that have one component or only one content focus, build data frame of 
            #allowable strand/substrand/content focus/component combinations
            if len(component_list) == 1 or len(content_focus_list) == 1:
                row, error_test, component_holding, content_focus_components_df = parse_line(
                    line, tagging_df, content_focus_list, component_list, error_test, component_holding, 
                    content_focus_components_df, initial components)
                if error_test == True:
                    error_data.append(row)                
                else: parsed_data.append(row)
                if component_holding == True:
                    component_holding.append(line)
            else: component_holding.append(line)
    initial_components = False
    #process items sent to component_holding
    return parsed_data, error_data, component_holding

In [262]:
def parse_line(line, tagging_df, content_focus_list, component_list, error_test, component_holding, content_focus_components_df, initial_components):
    row = {}
    strand_dict = {}
    strand_list = []
    differentiator_list = []
    content_focus_components_df = pd.DataFrame(index=index, columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
    
    
    InternalID = int(line['Internal ID'])

    Strand = line['Strand'].split("; ")
    for strand in Strand:
        strand_sub_dict = {}
        content_focus_df = tagging_df['Content_Focus']
        GUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
        strand_sub_dict = {'Strand': strand, 'GUID' : GUID, 'Substrand': []}
        strand_list.append(strand_sub_dict)
    if error_test == False:
        substrand_list = line['Sub-Strand'].split("; ")
        strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)
        if error_test == False:
            content_focus_list = line['Content Focus'].split("; ")
            strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)
            if error_test == False:
                differentiator_keys = ['Shape', 'Units', 'Steps']            
                for differentiator in differentiator_keys:
                    differentiator_sub_list = []
                    differentiator_df = tagging_df[differentiator]
                    differentiator_values = line[differentiator].split("; ")
                    for value in differentiator_values:
                        differentiator_sub_dict = {}
                        GUID, error_test = get_GUID_lite(value, differentiator_df, error_test)
                        differentiator_sub_dict = {differentiator: value, 'GUID': GUID}
                        differentiator_sub_list.append(differentiator_sub_dict)
                        if error_test == True:
                            break
                    differentiator_list.append(differentiator_sub_list)
    row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": differentiator_list}
    return row, error_test

In [196]:
data = []
data, error_data, component_holding = process_file(filename, tagging_dict)
#pprint.pprint(bad_data)

In [175]:
len(bad_data)
#pprint.pprint(bad_data)

45

In [176]:
#pprint.pprint(bad_data)
print len(component_holding)
print len(data)

71
1338


In [177]:
#pprint.pprint(data)

In [198]:
def processSubstrands(strand_list, substrand_list, df, error_test):
    GUID = None
    for substrand in substrand_list:
        if len(df[(df['Title']==lowercase(substrand))]) == 0:  
            error_test = True
            error_message = 'Error: Substrand ' + substrand + ' not found in master list'
            strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': error_message})
            break
    if len(substrand_list) < len(strand_list):
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': 'Error: more strands than substrands'})
    elif len(strand_list) == 1: 
        for substrand in substrand_list:
            strand_list, error_test = getSubstrandGUID(strand_list[0]['Strand'], strand_list, substrand, df, error_test)    
    elif len(substrand_list) == 2 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 2)
    elif len(substrand_list) == 3 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 3)
    else: 
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': 'Error: Multiple substrands, cannot parse yet'})
    return strand_list, error_test

#Build a data frame to track match status to determine whether strand-substrand pairings can be deduced    
def matchStrandsSubstrands(strand_list, substrand_list, df, error_test, matchlimit):
    strand_names = []
    for strand in strand_list:
        strand_names.append(strand['Strand'])

    match_status = pd.DataFrame(False,index=strand_names,columns=substrand_list)

    for strand in strand_names:
        for substrand in substrand_list:
            if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
                match_status.ix[strand,substrand] = True
    
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        for pair in true_pairs:
            strand_list, error_test = getSubstrandGUID(pair[0], strand_list, pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'GUID': 'Error: Ambiguous strand/substrand relationships'})
        error_test = True
    return strand_list, error_test

def getSubstrandGUID(strand, strand_list, substrand, df, error_test): 
    if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
        SubstrandGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand)), 'Unique ID'].item()
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append({'Substrand': substrand, 'GUID': SubstrandGUID, 'Content Focus': []})
    else: 
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append({'Substrand': substrand, 'GUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

    

In [202]:
strand_raw = "Data, Statistics, and Probability; Numbers and Operations"
substrand_raw = "Data; Operations"
content_focus_raw = "Interpret data displays; Calculate"
#print strand_raw
#print substrand_raw

error_test = False
strand_dict = {}
strand_list = []
temp_list = strand_raw.split("; ")
#print temp_list
for strand in temp_list:
    strand_sub_dict = {}
    #content_focus_df = tagging_df['Content_Focus']
    StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
    strand_sub_dict = {'Strand': strand, 'GUID' : StrandGUID, 'Substrand': []}
    strand_list.append(strand_sub_dict)

substrand_list = substrand_raw.split("; ")
strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)

content_focus_list = content_focus_raw.split('; ')
strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)

print strand_list


[{'Substrand': [{'Substrand': 'Data', 'GUID': '14D66ABA-8C17-11E6-81DC-467029C466BA', 'Content Focus': [{'GUID': 'C9DB3624-8C1D-11E6-867F-EA7829C466BA', 'Component': [], 'Content Focus': 'Interpret data displays'}]}], 'GUID': 'D66C3DCE-8BFB-11E6-921B-89CC29C466BA', 'Strand': 'Data, Statistics, and Probability'}, {'Substrand': [{'Substrand': 'Operations', 'GUID': '168B2CF6-8C17-11E6-81DC-467029C466BA', 'Content Focus': [{'GUID': 'CBFF09D0-8C1D-11E6-867F-EA7829C466BA', 'Component': [], 'Content Focus': 'Calculate'}]}], 'GUID': 'D66EB2C0-8BFB-11E6-921B-89CC29C466BA', 'Strand': 'Numbers and Operations'}]


In [215]:
def processContentFocus(strand_list, substrand_list, content_focus_list, df, error_test):
    GUID = None
    print content_focus_list
    for content_focus in content_focus_list:
        if len(df[(df['Title']==lowercase(content_focus))]) == 0:  
            error_test = True
            error_message = 'Error: Content Focus ' + content_focus + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': error_message})
            break
    if len(content_focus_list) < len(substrand_list):
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': 'Error: more Substrands than Content Focus terms'})
    elif len(substrand_list) == 1: 
        print content_focus_list
        for content_focus in content_focus_list:
            print content_focus
            strand_list, error_test = getContentFocusGUID(strand_list[0]['Strand'], strand_list, substrand_list[0], content_focus, df, error_test)    
    elif len(substrand_list) == len(content_focus_list):
        strand_list, error_test = matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, len(content_focus_list))
    else: 
        error_test = True
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': 'Error: Multiple content Focus terms, cannot parse yet'})
    return strand_list, error_test

def matchSubstrandsContentFocus(strand_list, content_focus_list, df, error_test, matchlimit):
    strand_substrand_names =[]
    print content_focus_list
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + '; ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=content_focus_list)
    print match_status
    for strand_substrand in strand_substrand_names:
        for content_focus in content_focus_list:
            #print strand_substrand
            #print content_focus
            if len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1]))& (df['Title']==lowercase(content_focus))]) == 1:
                match_status.ix[strand_substrand, content_focus] = True

    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test = getContentFocusGUID(pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1], pair[1], df, error_test)
    elif int(match_status.sum().sum()) < matchlimit:
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': 'Error: One or more content focus terms not found'})
        error_test = True
    else: 
        strand_list[0]['Substrand'][0]['Content Focus'].append({'Content Focus': content_focus, 'GUID': 'Error: Ambiguous strand/substrand/content focus relationships'})
        error_test = True
    return strand_list, error_test

def getContentFocusGUID(strand, strand_list, substrand, content_focus, df, error_test): 
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate(strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(content_focus))]) == 1:
        ContentFocusGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(content_focus)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append({'Content Focus': content_focus, 'GUID': ContentFocusGUID, 'Component': []})
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'].append({'Content Focus': content_focus, 'GUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

In [204]:
def processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, component_hold):
    GUID = None
    for component in component_list:
        if len(df[(df['Title']==lowercase(component))]) == 0:  
            error_test = True
            error_message = 'Error: Component ' + component + ' not found in master list'
            strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'GUID': error_message})
            break
    if len(content_focus_list) == 1:
        for component in component_list:
            strand_list, error_test, component_hold, guid = getComponentGUID(strand_list[0]['Strand'], strand_list, substrand_list[0], content_focus_list[0], component, df, error_test, component_hold) 
    elif len(substrand_list) == len(component_list):
        strand_list, error_test, component_hold = matchInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, len(component_list))            
    
    
    else: 
        component_hold = True
        strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'GUID': 'Component Hold'})
    return strand_list, error_test, component_hold

In [242]:
def getComponentGUID(strand, strand_list, substrand, content_focus, component, df, error_test, component_hold): 
    ComponentGUID = ""
    strand_index = next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)
    substrand_index = next(index for (index, d) in enumerate(strand_list[strand_index]['Substrand']) if d["Substrand"] == substrand)    
    content_focus_index = next(index for (index, d) in enumerate(strand_list[strand_index]['Substrand'][substrand_index]['Content Focus']) if d["Content Focus"] == content_focus)
    if len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': Component, 'GUID': ComponentGUID})
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Example Content Focus']==lowercase(content_focus)) & (df['Title']==lowercase(component))]) == 1:
        ComponentGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component)), 'Unique ID'].item()
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': Component, 'GUID': ComponentGUID})
    elif len(df[(df['Strand']==lowercase(strand)) & (df['Sub-Strand']==lowercase(substrand)) & (df['Title']==lowercase(component))]) > 1:
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': component, 'GUID': 'Holding'})
        component_hold = True
    else: 
        strand_list[strand_index]['Substrand'][substrand_index]['Content Focus'][content_focus_index]['Component'].append({'Component': component, 'GUID': 'Error: No unique Strand/Substrand/Component match found in master list'})
        error_test = True
    return strand_list, error_test, component_hold, ComponentGUID

In [259]:
def matchInitialComponent(strand_list, substrand_list, content_focus_list, component_list, df, error_test, matchlimit, content_focus_components_df):
    strand_substrand_names =[]
    for strand in strand_list:
        strand_name = strand['Strand']
        for substrand in strand['Substrand']:
            strand_substrand_names.append(strand_name + '; ' + substrand['Substrand'])

    match_status = pd.DataFrame(False,index=strand_substrand_names,columns=component_list)
    print match_status
    strand_substrand_names[0].split('; ')[0]

    for strand_substrand in strand_substrand_names:
        #Need to use substrand and also pull strand from strand list, might be easier to build from strand, down
        for component in component_list:
            initial_match_length = len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1]))& (df['Title']==lowercase(component))])
            if  initial_match_length == 1:
                match_status.ix[strand_substrand, component] = True
            elif initial_match_length > 1:
                    if len(df[(df['Strand']==lowercase(strand_substrand.split('; ')[0])) & (df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1])) & (df['Example Content Focus']==lowercase(content_focus_list[0])) & (df['Title']==lowercase(component))]) == 1:
                        match_status.ix[strand_substrand, component] = True
                
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        #print true_pairs
        for pair in true_pairs:
            strand_list, error_test, GUID = getComponentGUID(pair[0].split('; ')[0], strand_list, pair[0].split('; ')[1], content_focus_list[0], pair[1], df, error_test, component_hold)
            if len(content_focus_components_df[(content_focus_components_df['Strand']==lowercase(strand_substrand.split('; ')[0])) & (content_focus_components_df['Sub-Strand']==lowercase(strand_substrand.split('; ')[1])) & (content_focus_components_df['Content Focus']==lowercase(content_focus_list[0])) & (content_focus_components_df['Component']==lowercase(component))]) == 0:
                #add group to content_focus_gomponent_df
                df_add = {'Strand': lowercase(strand), 'Sub-Strand': lowercase(substrand), 'Content Focus': lowercase(content_focus), "Component": lowercase(component), "ComponentGUID": "test"}
                content_focus_components_df = content_focus_components_df.append(df_add, ignore_index = True)

    else: 
        strand_list[0]['Substrand'][0]['Content Focus'][0]['Component'].append({'Component': component, 'GUID': 'Error: Aha!'})
        error_test = True
    return strand_list, error_test, component_hold

In [241]:
strands = ['Numbers and Operations']
substrand_list = ['Operations']
content_focus_list = ['Calculate']
component_list = ['Addition']
error_test = False
component_hold = False
#strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)

strand_list=[]

for strand in strands:
    strand_sub_dict = {}
    #content_focus_df = tagging_df['Content_Focus']
    StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
    strand_sub_dict = {'Strand': strand, 'GUID' : StrandGUID, 'Substrand': []}
    strand_list.append(strand_sub_dict)
                        
strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)

strand_list, error_test = processContentFocus(strand_list, substrand_list, content_focus_list, content_focus_df, error_test)
pprint.pprint(strand_list)
strand_list, error_test, component_hold = processInitialComponent(strand_list, substrand_list, content_focus_list, component_list, component_df, error_test, component_hold)

pprint.pprint(strand_list[0])

['Calculate']
['Calculate']
Calculate
[{'GUID': 'D66EB2C0-8BFB-11E6-921B-89CC29C466BA',
  'Strand': 'Numbers and Operations',
  'Substrand': [{'Content Focus': [{'Component': [],
                                    'Content Focus': 'Calculate',
                                    'GUID': 'CBFF09D0-8C1D-11E6-867F-EA7829C466BA'}],
                 'GUID': '168B2CF6-8C17-11E6-81DC-467029C466BA',
                 'Substrand': 'Operations'}]}]
{'GUID': 'D66EB2C0-8BFB-11E6-921B-89CC29C466BA',
 'Strand': 'Numbers and Operations',
 'Substrand': [{'Content Focus': [{'Component': [{'Component': 'Addition',
                                                  'ComponentGUID': 'Error: No unique Strand/Substrand/Component match found in master list'}],
                                   'Content Focus': 'Calculate',
                                   'GUID': 'CBFF09D0-8C1D-11E6-867F-EA7829C466BA'}],
                'GUID': '168B2CF6-8C17-11E6-81DC-467029C466BA',
                'Substrand': 'Operation

In [258]:
content_focus_components_df = pd.DataFrame(columns=['Strand', 'Sub-Strand', 'Content Focus', 'Component', 'ComponentGUID'])
strand = 'Numbers and Operations'
substrand = 'Operations'
content_focus = 'Calculate'
component = 'Addition'


test = {'Strand': strand, 'Sub-Strand': substrand, 'Content Focus': content_focus, "Component": component, "ComponentGUID": "test"}
content_focus_components_df = content_focus_components_df.append(test, ignore_index = True)

if len(content_focus_components_df[(content_focus_components_df['Strand']==strand) & (content_focus_components_df['Sub-Strand']==substrand) & (content_focus_components_df['Content Focus']==content_focus) & (content_focus_components_df['Component']==component)]) == 0:
    #add group to content_focus_gomponent_df
    df_add = {'Strand': strand, 'Sub-Strand': substrand, 'Content Focus': content_focus, "Component": component, "ComponentGUID": "test"}
    content_focus_components_df = content_focus_components_df.append(df_add, ignore_index = True)

component = 'Subtraction'
test = {'Strand': strand, 'Sub-Strand': substrand, 'Content Focus': content_focus, "Component": component, "ComponentGUID": "test"}


if len(content_focus_components_df[(content_focus_components_df['Strand']==strand) & (content_focus_components_df['Sub-Strand']==substrand) & (content_focus_components_df['Content Focus']==content_focus) & (content_focus_components_df['Component']==component)]) == 0:
    #add group to content_focus_gomponent_df
    df_add = {'Strand': strand, 'Sub-Strand': substrand, 'Content Focus': content_focus, "Component": component, "ComponentGUID": "test"}
    content_focus_components_df = content_focus_components_df.append(df_add, ignore_index = True)


print content_focus_components_df

                   Strand  Sub-Strand Content Focus    Component ComponentGUID
0  Numbers and Operations  Operations     Calculate     Addition          test
1  Numbers and Operations  Operations     Calculate  Subtraction          test


In [257]:
if len(content_focus_components_df[(content_focus_components_df['Strand']==strand) & (content_focus_components_df['Sub-Strand']==substrand) & (content_focus_components_df['Content Focus']==content_focus) & (content_focus_components_df['Component']==component)]) == 0:
    #add group to content_focus_gomponent_df
    df_add = {'Strand': strand, 'Sub-Strand': substrand, 'Content Focus': content_focus, "Component": component, "ComponentGUID": "test"}
    content_focus_components_df = content_focus_components_df.append(df_add, ignore_index = True)


print content_focus_components_df

                   Strand  Sub-Strand Content Focus    Component ComponentGUID
0  Numbers and Operations  Operations     Calculate     Addition          test
1  Numbers and Operations  Operations     Calculate     Addition          test
2  Numbers and Operations  Operations     Calculate  Subtraction          test
3  Numbers and Operations  Operations     Calculate  Subtraction          test


In [224]:
print component_df

    Asset Subject                              Example Content Focus  \
0     Mathematics                                    Classify angles   
1     Mathematics  Identify vertical, adjacent, complementary, su...   
2     Mathematics  Identify vertical, adjacent, complementary, su...   
3     Mathematics  Identify vertical, adjacent, complementary, su...   
4     Mathematics        Identify points, rays, lines, line segments   
5     Mathematics  Identify vertical, adjacent, complementary, su...   
6     Mathematics  Identify vertical, adjacent, complementary, su...   
7     Mathematics  Identify vertical, adjacent, complementary, su...   
8     Mathematics  Identify vertical, adjacent, complementary, su...   
9     Mathematics                    Know sums of angles in polygons   
10    Mathematics                                    Classify angles   
11    Mathematics  Identify vertical, adjacent, complementary, su...   
12    Mathematics                                    Classify an

In [188]:
def lowercase(text):
    return unicode(text, "latin2").lower()

In [189]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
    return data

In [190]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [191]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, 'Content_Focus': content_focus_df, 'Component': component_df}

In [192]:

def get_GUID_lite(tag, df, error_test): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        else: 
            GUID = "Error: no matches found"
            error_test = True
    else: 
        GUID = "Error: resolve multiple values"
        error_test = True
    return GUID, error_test

In [193]:
number_test = "Square roots"
error_test = False
print get_GUID_lite(number_test, number_df, error_test)

('1690A96C-97C3-11E6-88DC-EDD729C466BA', False)


In [194]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument

tag = 'N/A'
if tag == r'N/A':
    print tag


N/A


In [195]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
                        {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
            {Units: [Units1, ...]}
            {Steps: Steps}
            {Context: Context}
            {SIG Stem: [Sig Stem1, ...]}
            {SIG Options: [SIG Options1, ...]}
            {Additional Modifiers: [AM1, ...]}
}

'''

'\nData structure\n\n{Internal ID: value\n    {Organizers: [\n        {Strand: [ {Strand: Strand_value,\n                   Strand_GUID: GUID,\n                   Substrand: [{Substrand: Substrand_value,\n                                Substrand_GUID: GUID,\n                                Content_Focus: [{Content_Focus: Content_Focus_value,                       \n                                                Content_Focus_GUID: GUID,\n                                                Component: [{Component: Component_value,\n                                                              Component_GUID: GUID}]}]}]}]\n        {Descriptors: {Number: [N1, N2, ...]}\n                        {Digits: [D1, D2, ...]}\n            {Shape: [Shape1, ...]}\n            {Units: [Units1, ...]}\n            {Steps: Steps}\n            {Context: Context}\n            {SIG Stem: [Sig Stem1, ...]}\n            {SIG Options: [SIG Options1, ...]}\n            {Additional Modifiers: [AM1, ...]}\n}\n\n'